# Advanced NLP Assignment 3

## Q1

**(1)**
 Explain three different points between P-Tuning V1 and P-Tuning V2.
 

- **Prompt depth**: P-Tuning V1 only adds continuous prompts to the input layer of the pre-trained model, while P-Tuning V2 adds prompts to every layer of the model. This increases the capacity and impact of the prompts.
- **Classification head**: P-Tuning V1 uses a language modeling head to predict verbalizers, which are natural language tokens that correspond to labels. P-Tuning V2 uses a linear classifier on top of the [CLS] token or the target tokens, which is simpler and more compatible with sequence labeling tasks.
- **Reparameterization**: P-Tuning V1 always uses a reparameterization encoder such as an MLP to transform the trainable embeddings. P-Tuning V2 finds that the usefulness of reparameterization depends on the tasks and datasets, and sometimes a simple embedding layer is enough.

**(2)**
 Write code to finetune Tiny-BERT with P-Tuning v2 method using the MRPC Dataset.


In [1]:
# !python -m pip install --user --upgrade evaluate

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset, load_metric
from peft import get_peft_config, get_peft_model, PrefixTuningConfig, TaskType, PeftType

import torch
import transformers
import numpy as np
import evaluate
import warnings

warnings.simplefilter("ignore")

global_metric = load_metric("./accuracy/accuracy.py")

# Load the MRPC dataset
dataset = load_dataset(
    path = "json", 
    data_dir = "./Q1_MRPC_dataset", 
    data_files = {"train": "train.jsonl", "test": "test.jsonl", "validation": "validation.jsonl"}
)

# Load the Tiny-BERT model
model_dir = "./Q1_model_checkpoint_tinybert/"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
print(tokenizer)
model = AutoModelForSequenceClassification.from_pretrained(model_dir, num_labels=2)
# peft_config = PrefixTuningConfig(task_type="SEQ_CLS", num_virtual_tokens=10, prefix_projection=True)
peft_config = PrefixTuningConfig(task_type=TaskType.SEQ_CLS, num_virtual_tokens=5, prefix_projection=True)
model = get_peft_model(model, peft_config)

# Tokenize the dataset
def tokenize_function(example):
    return tokenizer(example["text1"], example["text2"], padding=True, truncation=True, max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Print the first tokenized example
# print(tokenized_dataset["train"][0])

# Define the optimizer and the learning rate scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
scheduler = transformers.get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(dataset["train"]) * 3)

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.71,
    push_to_hub=False,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
)

# Define the training and evaluation functions
def compute_metrics(eval_preds):
#     metric = load_metric("accuracy")
#     metric = evaluate.load("accuracy")
    metric = global_metric
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    print(logits)
    return metric.compute(predictions=predictions, references=labels)

def m_train():
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["validation"],
        data_collator=None,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )
    trainer.train()
    

# def m_evaluate():
#     trainer = Trainer(
#         model=model,
#         args=training_args,
#         train_dataset=None,
#         eval_dataset=tokenized_dataset["validation"],
#         data_collator=None,
#         tokenizer=tokenizer,
#         compute_metrics=compute_metrics,
#     )
#     result = trainer.evaluate()
#     return result
    
print(dataset["train"])
    
    
# Train the model
m_train()

# # Evaluate the model
# m_evaluate()


C:\Users\Wandering Troubadour\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./Q1_model_checkpoint_tinybert/ and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertTokenizerFast(name_or_path='./Q1_model_checkpoint_tinybert/', vocab_size=30522, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Dataset({
    features: ['text1', 'text2', 'label', 'idx', 'label_text'],
    num_rows: 3668
})


Step,Training Loss,Validation Loss,Accuracy
10,0.685400,0.660578,0.683824
20,0.660900,0.640993,0.683824
30,0.655100,0.639147,0.683824
40,0.644000,0.629369,0.683824
50,0.634100,0.624691,0.683824
60,0.621700,0.624080,0.683824
70,0.642800,0.625240,0.683824
80,0.660300,0.636651,0.683824
90,0.619300,0.623872,0.683824
100,0.600500,0.624581,0.683824


...

[[-0.3722142   0.40430856]
 [-0.37205458  0.40421715]
 [-0.37226236  0.40447396]
 [-0.3719557   0.40404707]
 [-0.37223262  0.40430143]
 [-0.37215006  0.40439576]
 [-0.37215889  0.40426064]
 [-0.37224105  0.40443057]
 [-0.37231207  0.40442824]
 [-0.37191033  0.40403026]
 [-0.37207657  0.4042726 ]
 [-0.37218195  0.40426356]
 [-0.37219474  0.40423965]
 [-0.37205154  0.40430534]
 [-0.37186393  0.4040578 ]
 [-0.3722843   0.40440303]
 [-0.37227812  0.40445906]
 [-0.37216082  0.40422884]
 [-0.37203273  0.40424007]
 [-0.37222746  0.40439194]
 [-0.37218425  0.40424782]
 [-0.37225     0.40444285]
 [-0.37213254  0.4043454 ]
 [-0.3721011   0.40434352]
 [-0.37225795  0.40449387]
 [-0.37232673  0.40452614]
 [-0.37179393  0.40390834]
 [-0.3719836   0.40410113]
 [-0.37184954  0.40404817]
 [-0.37222266  0.4044202 ]
 [-0.37224126  0.40441626]
 [-0.37205762  0.4042529 ]
 [-0.37194562  0.40407825]
 [-0.37227213  0.40440518]
 [-0.37211657  0.40427738]
 [-0.37199187  0.4041816 ]
 [-0.372091    0.40423965]
 

[[-0.46656972  0.5032007 ]
 [-0.4665205   0.5032629 ]
 [-0.46646476  0.50321674]
 [-0.46646357  0.5031346 ]
 [-0.46647757  0.50311947]
 [-0.46662855  0.50342506]
 [-0.46645182  0.5031094 ]
 [-0.46662295  0.50333   ]
 [-0.46661252  0.5032343 ]
 [-0.4663481   0.5030111 ]
 [-0.46641147  0.50313693]
 [-0.4665203   0.5031495 ]
 [-0.4665727   0.50316685]
 [-0.4664213   0.5032529 ]
 [-0.46616787  0.502956  ]
 [-0.4665063   0.5031994 ]
 [-0.46649975  0.5032298 ]
 [-0.46655148  0.5031603 ]
 [-0.46642357  0.50314486]
 [-0.4664505   0.5031602 ]
 [-0.46660858  0.5032597 ]
 [-0.46651924  0.5032276 ]
 [-0.4665075   0.5032901 ]
 [-0.4664924   0.5032829 ]
 [-0.46649158  0.5032788 ]
 [-0.46677178  0.50351626]
 [-0.46616727  0.5028536 ]
 [-0.46634585  0.50303316]
 [-0.46620607  0.5029445 ]
 [-0.4666763   0.5034264 ]
 [-0.46663886  0.5033933 ]
 [-0.4664585   0.5032381 ]
 [-0.46628428  0.5029706 ]
 [-0.4666618   0.5033119 ]
 [-0.46649718  0.5032003 ]
 [-0.46642047  0.50318104]
 [-0.46644086  0.50315714]
 

[[-0.4481671   0.48260128]
 [-0.44830728  0.48286006]
 [-0.44822037  0.48269477]
 [-0.44875586  0.48310694]
 [-0.44853276  0.48295885]
 [-0.4481309   0.48273185]
 [-0.44814342  0.48265418]
 [-0.4483788   0.48275015]
 [-0.44799614  0.4824498 ]
 [-0.4489408   0.48309526]
 [-0.4486633   0.48301363]
 [-0.4483828   0.48285857]
 [-0.4480319   0.48248   ]
 [-0.44820327  0.48276612]
 [-0.44886702  0.48316112]
 [-0.44827598  0.48274165]
 [-0.44843942  0.4828302 ]
 [-0.44806796  0.48253143]
 [-0.44852293  0.48282617]
 [-0.44774467  0.4822858 ]
 [-0.44845325  0.4829047 ]
 [-0.448281    0.4826329 ]
 [-0.44819146  0.48276058]
 [-0.4486853   0.48307738]
 [-0.44820076  0.48279878]
 [-0.44816822  0.48273277]
 [-0.44835413  0.48282126]
 [-0.44850475  0.4829212 ]
 [-0.44831115  0.48277113]
 [-0.44837457  0.48283502]
 [-0.44814718  0.48267764]
 [-0.44889045  0.48321688]
 [-0.44821954  0.4827528 ]
 [-0.44827944  0.4826695 ]
 [-0.44858938  0.48299053]
 [-0.44852626  0.48292857]
 [-0.44848937  0.48292276]
 

[[-0.35035738  0.37826622]
 [-0.35039616  0.3784724 ]
 [-0.35025123  0.37817782]
 [-0.35040432  0.3780564 ]
 [-0.3505016   0.3783717 ]
 [-0.35006374  0.37825722]
 [-0.3503952   0.378554  ]
 [-0.34986746  0.37748808]
 [-0.35011315  0.3782354 ]
 [-0.3499523   0.37692547]
 [-0.35048306  0.3778429 ]
 [-0.35072917  0.3788632 ]
 [-0.35008055  0.37827104]
 [-0.35019195  0.37811095]
 [-0.3501739   0.37738228]
 [-0.35017404  0.37813884]
 [-0.350252    0.37790838]
 [-0.35022658  0.37842584]
 [-0.35021126  0.3775667 ]
 [-0.34984607  0.37803277]
 [-0.35035458  0.37833023]
 [-0.35003757  0.37775064]
 [-0.35017085  0.37823075]
 [-0.34982696  0.37728238]
 [-0.3507174   0.37889552]
 [-0.34998322  0.37820533]
 [-0.35053778  0.37838835]
 [-0.35069478  0.37850666]
 [-0.35044637  0.37811998]
 [-0.35015357  0.37802377]
 [-0.3499151   0.37796748]
 [-0.35038662  0.3776064 ]
 [-0.35085106  0.37898004]
 [-0.34986034  0.377544  ]
 [-0.3505808   0.37816566]
 [-0.35019597  0.37775952]
 [-0.35028672  0.3780297 ]
 

KeyboardInterrupt: 

In [ ]:
def m_evaluate():
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=None,
        eval_dataset=tokenized_dataset["test"],
        data_collator=None,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )
    result = trainer.evaluate()
    return result

m_evaluate()

## Q2

Efficient finetuning with adapter method.

1. Explain the adapter-based finetuning. You can also refer to the Adapter